In [ ]:
import pandas as pd
import numpy as np
import pathlib
SEED = 19221991 #Initially random, yet replicable
SAMPLE_SIZE = int(100 / 4) # 100 total registrars to check
# Bikeshedding way to write to user directory
USER_DIR = str(pathlib.Path.home()) + '/'

It's too much work to check the payment methods for every single registrar represented in our set, so instead we'll take a stratified sample.
The set is already sorted by malicious_ratio descending. We'll make every quartile a stratum and then sample from that.

To make it easier to manually find these registrars' sites, we merge the ICANN list of currently accredited registrars, because the contact info includes a website. Otherwise, it's really hard to map legal names to brand names for many registrars, and even harder the other way around.

Not all of them are mapped, I believe the only ones missing are for deactivated registrars? Unconfirmed.

In [ ]:
df = pd.read_csv(USER_DIR+"most_malicious.csv")
registrars = pd.read_csv("/data/Accredited-Registrars-202412054424.csv")

size_before = len(df)
df = df.rename(columns={"registrar_id":"IANA Number"})
df = df.merge(registrars,on="IANA Number",how="left")
df.drop('Registrar Name', axis=1, inplace=True) # Redundant
assert(size_before == len(df)) # We shouldn't lose/gain any entries when merging
# Slots for manual info gathering we need
df['Supports Crypto'] = None
df['Free Services'] = None
#df

In [ ]:
strata = np.array_split(df,4) #List of 4 dataframes, probably not equally sized
print([len(stratum) for stratum in strata])

In [ ]:
samples = list()
for stratum in strata:
    samples.append(stratum.sample(n=SAMPLE_SIZE,random_state=SEED))
#samples[0]

In [ ]:
for i, sample in enumerate(samples):
    sample.to_csv(USER_DIR + 'stratum_'+str(i)+'_sample.csv')